<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Load-Dataset" data-toc-modified-id="Load-Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Dataset</a></span></li></ul></li><li><span><a href="#Regulon-enrichments" data-toc-modified-id="Regulon-enrichments-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Regulon enrichments</a></span></li><li><span><a href="#Functional-Enrichments" data-toc-modified-id="Functional-Enrichments-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functional Enrichments</a></span></li><li><span><a href="#Combine-Annotations" data-toc-modified-id="Combine-Annotations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combine Annotations</a></span></li></ul></div>

In [1]:
import sys, os
sys.path.append('../../scripts/')
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.metrics import r2_score
from itertools import combinations
import numpy as np
from scipy import stats
from enrichments import *
from core import *

# Setup

## Load Dataset

In [2]:
datasets = sorted([x for x in os.listdir(os.path.join(DATA_DIR,'iModulons/'))
            if '.' not in x])

In [3]:
# Thresholds were obtained from 2_identify_thresholds
cutoffs = {'MA-1': 550,
 'MA-2': 600,
 'MA-3': 350,
 'RNAseq-1': 700,
 'RNAseq-2': 300,
 'combined': 400}

In [4]:
class IcaData():
    def __init__(self,M,A,X,metadata,cutoff):
        self.M = pd.read_csv(M,index_col=0)
        self.M.columns = self.M.columns.astype(int)
        self.A = pd.read_csv(A,index_col=0)
        self.A.index = self.A.index.astype(int)
        self.X = pd.read_csv(X,index_col=0)
        self.metadata = pd.read_csv(metadata,index_col=0)
        self.thresholds = {k:self.get_threshold(k,cutoff) for k in self.M.columns}
    
    def show_enriched(self,k):
        gene_table = gene_info.copy()
        gene_table = gene_table.reindex(self.M.index)
        in_imod = abs(self.M[k]) > self.thresholds[k]
        weights = self.M.loc[in_imod,k]
        weights.name = 'weight'
        rows = gene_table.loc[in_imod]
        final_rows = pd.concat([weights,rows],axis=1)
        return final_rows.sort_values('weight')
    
    def get_threshold(self,k,cutoff):
        i=0
        genes = abs(self.M[k]).sort_values()
        k2,p = stats.normaltest(self.M[k])
        while k2 > cutoff:
            i -= 1
            k2,p = stats.normaltest(self.M.loc[genes.index[:i],k])
        imod_genes = genes.iloc[i:]
        return np.mean([genes.iloc[i], genes.iloc[i - 1]])

def load(dataset):
    # Define directories
    ds_dir = os.path.join(DATA_DIR,'iModulons',dataset)
    
    # Define files
    X_file = os.path.join(DATA_DIR,'processed_data',dataset+'.csv')
    M_file = os.path.join(ds_dir,'M.csv')
    A_file = os.path.join(ds_dir,'A.csv')
    metadata_file = os.path.join(DATA_DIR,'metadata',dataset+'_metadata.csv')
    
    return IcaData(M_file,A_file,X_file,metadata_file,cutoffs[dataset])

In [5]:
# Load datasets
objs = {}
for ds in tqdm(datasets):
    objs[ds] = load(ds)

In [6]:
summary_table = pd.DataFrame(dtype=int)
for ds in datasets:
    summary_table.loc[ds,'Samples'] = len(objs[ds].metadata)
    summary_table.loc[ds,'Conditions'] = len(objs[ds].metadata[['project_id','condition_id']].drop_duplicates())
    summary_table.loc[ds,'I-modulons'] = len(objs[ds].A.index)
    
    r2_list = []
    for name,group in objs[ds].metadata.groupby(['project_id','condition_id']):
        for i1,i2 in combinations(group.index,2):
            r2_list.append(r2_score(objs[ds].X[i1],objs[ds].X[i2]))
    
    summary_table.loc[ds,'Mean R2'] = np.mean(r2_list)
summary_table

,Samples,Conditions,I-modulons,Mean R2
MA-1,260.0,115.0,103.0,0.970481
MA-2,124.0,39.0,58.0,0.935839
MA-3,56.0,20.0,32.0,0.984067
RNAseq-1,278.0,163.0,91.0,0.974502
RNAseq-2,84.0,28.0,52.0,0.953760
combined,802.0,365.0,181.0,0.962257


# Regulon enrichments

In [7]:
import warnings
warnings.simplefilter('ignore',category=FutureWarning)

In [8]:
def get_enrichments(ica_data):
    all_genes = set(ica_data.M.index)
    list2struct = []
    
    # Get TF enrichments for each component
    for k in tqdm(ica_data.M.columns):
        genes = set(ica_data.show_enriched(k).index)
        df = compute_enrichments(genes,all_genes,trn,fdr=1e-5)
        df['component'] = k
        df.index.name = 'description'
        df = df.reset_index(drop=False)
        list2struct.append(df)
        
    DF_enrich = pd.concat(list2struct).reset_index(drop=True)
    
    return DF_enrich

In [9]:
list2struct = []
for ds in datasets:
    df_enrich = get_enrichments(objs[ds])
    df_enrich['dataset'] = ds
    list2struct.append(df_enrich)

DF_all_enrich = pd.concat(list2struct).reset_index(drop=True)

In [10]:
DF_reg_enrich = DF_all_enrich.sort_values(['dataset','component',
                                            'pvalue','precision']).reset_index(drop=True)
DF_reg_enrich['type'] = 'regulatory'

In [11]:
col_order = ['dataset','component','type','description','pvalue','qvalue','precision','recall','f1score','TP']
DF_reg_enrich = DF_reg_enrich[col_order]

In [12]:
DF_reg_enrich.head()

,dataset,component,type,description,pvalue,qvalue,precision,recall,f1score,TP
0,MA-1,0,regulatory,arcA,2.993701e-12,8.232678e-10,0.900000,0.055556,0.104651,9.0
1,MA-1,0,regulatory,fur,1.497576e-11,2.059167e-09,0.800000,0.072072,0.132231,8.0
2,MA-1,0,regulatory,fnr,2.574975e-08,2.360394e-06,0.800000,0.028674,0.055363,8.0
3,MA-1,0,regulatory,crp,4.685148e-08,3.221039e-06,0.900000,0.019149,0.037500,9.0
4,MA-1,1,regulatory,fnr,2.368387e-19,6.513065e-17,0.538462,0.100358,0.169184,28.0


In [13]:
DF_reg_enrich.drop_duplicates(['dataset','component']).to_csv(os.path.join(DATA_DIR,'iModulons','1TF_enrichments.csv'))

In [14]:
DF_reg_enrich.to_csv(os.path.join(DATA_DIR,'iModulons','all_reg_enrichments.csv'))

# Functional Enrichments

In [15]:
DF_GO = pd.read_csv(os.path.join(GENE_DIR,'DF_GO.csv'),index_col=0)

In [16]:
go_dict = {}
for name,group in DF_GO.groupby('go_name'):
    genes = set(group.bnumber)
    go_dict[name] = genes

In [17]:
def get_go_enrichments(ica_data):
    all_genes = set(ica_data.M.index)
    enrich_list = []
    for k in tqdm(ica_data.M.columns):
        ic_genes = set(ica_data.show_enriched(k).index)
        
        list2struct = []
        for go,go_genes in go_dict.items():
            ((tp,fp),(fn,tn)) = contingency(go_genes,ic_genes,all_genes)
            odds,pval = stats.fisher_exact(((tp,fp),(fn,tn)),alternative='greater')
            if len(ic_genes) > 0:
                list2struct.append([go,k,tp,tp/len(ic_genes),tp/len(go_genes),pval])
            else:
                list2struct.append([go,k,tp,0,0,pval])
        df_pvals = pd.DataFrame(list2struct,columns = ['description','component','TP',
                                                       'precision','recall','pvalue'])
        enrich_list.append(FDR(df_pvals,.01))
    go_enrichments = pd.concat(enrich_list).reset_index(drop=True)
    go_enrichments.component = go_enrichments.component.astype(int)
    return go_enrichments

In [18]:
list2struct = []

for ds in datasets:
    df = get_go_enrichments(objs[ds])
    df['dataset'] = ds
    list2struct.append(df)
    
DF_all_go = pd.concat(list2struct).reset_index(drop=True)

In [19]:
DF_go_enrich = DF_all_go.sort_values(['dataset','component',
                                      'pvalue'])
DF_go_enrich['f1score'] = 2*DF_go_enrich.precision*DF_go_enrich.recall/(DF_go_enrich.precision+DF_go_enrich.recall)

In [20]:
DF_go_enrich['type'] = 'functional'
col_order = ['dataset','component','type','description','pvalue','qvalue','precision','recall','f1score','TP']
DF_go_enrich = DF_go_enrich[col_order]

In [21]:
DF_go_enrich.head()

,dataset,component,type,description,pvalue,qvalue,precision,recall,f1score,TP
0,MA-1,0,functional,tricarboxylic acid cycle,1.024288e-10,3.410877e-07,0.5,0.333333,0.400000,5
1,MA-1,0,functional,aerobic respiration,2.308688e-07,3.843966e-04,0.4,0.166667,0.235294,4
2,MA-1,0,functional,electron transfer activity,3.449666e-06,3.829130e-03,0.4,0.086957,0.142857,4
3,MA-1,0,functional,oxoglutarate dehydrogenase complex,5.979860e-06,3.982587e-03,0.2,1.000000,0.333333,2
4,MA-1,0,functional,succinate-CoA ligase complex (ADP-forming),5.979860e-06,3.982587e-03,0.2,1.000000,0.333333,2


In [22]:
DF_go_enrich.drop_duplicates(['dataset','component']).to_csv(os.path.join(DATA_DIR,'iModulons','GO_enrichments.csv'))

In [23]:
DF_go_enrich.to_csv(os.path.join(DATA_DIR,'iModulons','all_GO_enrichments.csv'))

# Combine Annotations

In [24]:
DF_reg_enrich = pd.read_csv(os.path.join(DATA_DIR,'iModulons','1TF_enrichments.csv'),index_col=0)
DF_go_enrich = pd.read_csv(os.path.join(DATA_DIR,'iModulons','GO_enrichments.csv'),index_col=0)

In [25]:
# Combine regulatory and GO enrichments
DF_categories = pd.concat([DF_reg_enrich,DF_go_enrich]).sort_values(['dataset','component','qvalue'])

# Add names
DF_categories['name'] = DF_categories.dataset.str.cat(DF_categories.component.astype(str),sep='_')

# Filter out GO enrichments when regulatory enrichment exists
DF_categories = DF_categories.sort_values(['dataset','component','type','qvalue'],ascending=[1,1,0,1]).drop_duplicates(['dataset','component'])

# Reorganize columns
col_order = ['dataset','component','type','description','pvalue','qvalue','precision','recall','f1score','TP']
DF_categories = DF_categories.set_index('name')[col_order]

# Add empty rows for uncharacterized comps
other_comps = []
idx_list = []
for ds in datasets:
    for k in objs[ds].M.columns:
        idx = ds+'_'+str(k)
        if idx not in DF_categories.index:
            idx_list.append(idx)
            other_comps.append([ds,k,'uncharacterized']+[None]*7)
DF_other = pd.DataFrame(other_comps,columns=DF_categories.columns,index=idx_list)

DF_categories = pd.concat([DF_categories,DF_other]).sort_values(['dataset','component'])

In [26]:
DF_categories.to_csv(os.path.join(DATA_DIR,'iModulons','categories.csv'))